### Attributes list for reference

    - Inner:   '5_o_Clock_Shadow', 
    - Inner:   'Arched_Eyebrows',
    - None:    'Attractive',
    - Inner:   'Bags_Under_Eyes',
    - Outer:   'Bald',
    - Outer:   'Bangs',
    - Inner:   'Big_Lips',
    - Inner:   'Big_Nose',
    - Outer:   'Black_Hair',
    - Outer:   'Blond_Hair',
    - None:    'Blurry',
    - Outer:   'Brown_Hair',
    - Inner:   'Bushy_Eyebrows',
    - Outer:   'Chubby',
    - Outer:   'Double_Chin',
    - Inner:   'Eyeglasses',
    - Inner:   'Goatee',
    - Outer:   'Gray_Hair',
    - Inner:   'Heavy_Makeup',
    - Inner:   'High_Cheekbones',
    - Inner:   'Male',
    - Inner:   'Mouth_Slightly_Open',
    - Inner:   'Mustache',
    - Inner:   'Narrow_Eyes',
    - Inner:   'No_Beard',
    - Outer:   'Oval_Face',
    - Inner:   'Pale_Skin',
    - Inner:   'Pointy_Nose',
    - Outer:   'Receding_Hairline',
    - Inner:   'Rosy_Cheeks',
    - Outer:   'Sideburns',
    - Inner:   'Smiling',
    - Outer:   'Straight_Hair',
    - Outer:   'Wavy_Hair',
    - Outer:   'Wearing_Earrings',
    - Outer:   'Wearing_Hat',
    - Inner:   'Wearing_Lipstick',
    - Outer:   'Wearing_Necklace',
    - Outer:   'Wearing_Necktie',
    - Inner:   'Young'

# Step 0. Create list_eval_partition.txt.

In [38]:
total_images = 30000
trainset_size = 24000
testset_size = 6000
evalset_size = 0

split_file_path = '/home/joshua/Projects/dataset/CelebAMask-HQ/annotations/list_eval_partition.txt'

with open(split_file_path, 'w') as f:
    for i in range(total_images):
        image_name = f'{i}.jpg'
        if i < trainset_size:
            partition = 0  # Trainset
        elif trainset_size <= i < trainset_size+testset_size:
            partition = 1  # Testset
        else:
            partition = 2  # evalset
        f.write(f'{image_name} {partition}\n')

# Step 1. Load MobileNetV2 model

In [1]:
import torch
import os
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
from PIL import Image
from torchvision.models import mobilenet_v2
import pandas as pd
from sklearn.model_selection import train_test_split

# 获取MobileNetV2模型和预处理函数
MobileNetV2 = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

# 设置属性标签数量
num_attributes = 40

# 构建模型
base_model = mobilenet_v2(pretrained=True)
num_features = base_model.classifier[1].in_features
base_model.classifier = nn.Linear(num_features, num_attributes)

# # 固定其他层的参数
# for param in base_model.parameters():
#     param.requires_grad = False

# # 将最后一层的参数设置为可训练
# for param in base_model.classifier.parameters():
#     param.requires_grad = True

path_to_load_model = '/home/joshua/Projects/FALCO/Checkpoints/model_celebAHQ_2023-05-28_19-37-50.pt'
base_model.load_state_dict(torch.load(path_to_load_model))

model = base_model.cuda()
model

Using cache found in /home/joshua/.cache/torch/hub/pytorch_vision_v0.10.0


MobileNetV2(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05,

# Step 2. Load Data

In [2]:
# Path
image_folder = '/home/joshua/Projects/dataset/CelebAMask-HQ/data'
split_file = '/home/joshua/Projects/dataset/CelebAMask-HQ/annotations/list_eval_partition.txt'
attribute_file = '/home/joshua/Projects/dataset/CelebAMask-HQ/annotations/CelebAMask-HQ-attribute-anno.txt'

In [3]:
attribute_df = pd.read_csv(attribute_file, sep=' ', skiprows=[0, 1], header=None, engine='python')  # 跳过前两行
attribute_df = attribute_df.replace(-1, 0)
attribute_df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.jpg,NaN,0,1,1,1,0,0,1,0,...,0,1,0,1,0,0,1,0,0,1
1,1.jpg,NaN,0,1,1,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,1
2,2.jpg,NaN,0,0,1,1,0,0,1,0,...,0,1,0,1,1,0,1,0,0,1
3,3.jpg,NaN,0,0,1,0,0,0,0,1,...,0,1,0,1,0,0,1,0,0,1
4,4.jpg,NaN,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29995.jpg,NaN,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
29996,29996.jpg,NaN,1,0,0,1,0,0,0,1,...,1,1,0,0,0,0,0,0,0,1
29997,29997.jpg,NaN,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
29998,29998.jpg,NaN,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1


In [4]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 自定义数据集类
class CreateDataset(Dataset):
    def __init__(self, image_folder, attribute_file, transform=None):
        self.image_folder = image_folder
        self.attribute_df = pd.read_csv(attribute_file, sep=' ', skiprows=[0, 1], header=None, engine='python')  # 跳过前两行
        self.attribute_df = self.attribute_df.replace(-1, 0)
        self.transform = transform

    def __len__(self):
        return len(self.attribute_df)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_folder, self.attribute_df.iloc[idx, 0])
        image = Image.open(image_path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)

        attribute_labels = self.attribute_df.iloc[idx, 2:].values.astype(int)
        return image, attribute_labels


# 加载图片数据集
dataset = CreateDataset(image_folder, attribute_file, transform=preprocess)

# 划分训练集和测试集
# train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# 创建训练集和测试集的DataLoader
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)


# Step 3. Fit model

In [5]:
# 定义损失函数和优化器
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=0.75):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        bce_loss = nn.BCEWithLogitsLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        return focal_loss.mean()
    
    
criterion = FocalLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    
    for images, labels in train_dataloader:
        images = images.cuda()
        labels = labels.cuda()
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels.float())
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 统计训练损失和准确率
        train_loss += loss.item() * images.size(0)
        train_correct += (outputs > 0.5).eq(labels > 0.5).sum().item()
    
    # 打印训练信息
    train_loss = train_loss / len(dataset)
    train_acc = train_correct / len(dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')

Epoch [1/10], Train Loss: 0.0252, Train Accuracy: 37.3761
Epoch [2/10], Train Loss: 0.0239, Train Accuracy: 37.3753
Epoch [3/10], Train Loss: 0.0234, Train Accuracy: 37.4097
Epoch [4/10], Train Loss: 0.0228, Train Accuracy: 37.4687
Epoch [5/10], Train Loss: 0.0222, Train Accuracy: 37.5315
Epoch [6/10], Train Loss: 0.0217, Train Accuracy: 37.5788
Epoch [7/10], Train Loss: 0.0211, Train Accuracy: 37.6493
Epoch [8/10], Train Loss: 0.0205, Train Accuracy: 37.7089
Epoch [9/10], Train Loss: 0.0200, Train Accuracy: 37.7649
Epoch [10/10], Train Loss: 0.0194, Train Accuracy: 37.8221


In [6]:
# 保存模型
import datetime

# 获取当前日期和时间
current_time = datetime.datetime.now()
path_to_save_model = 'Checkpoints/model_celebAHQ_'+f"{current_time.strftime('%Y-%m-%d_%H-%M-%S')}"+'.pt'

torch.save(model.state_dict(), path_to_save_model)